# ERP responses.

In [ ]:
import os
import glob
import numpy as np
import matplotlib.pyplot as plt
import mne
from mne.stats import permutation_cluster_1samp_test, permutation_cluster_test
from scipy import stats

### Parameters.

In [ ]:
reject = None #{'eeg':100e-6}
tmin, tmax = (-0.05, 0.55)
sfreq = 128
colors = ['#1f77b4', '#ff7f0e']
n_bins = 5

### Import data.

In [ ]:
# Behavior (sequences)
dname_seq = '/media/jacques/DATA1/2019_MusicPred/experimentContinuousMatrix/data/behavior/'
DEC = np.load(dname_seq + 'DEC.npy')
MUSICIANS = np.load(dname_seq + 'MUSICIANS.npy')
SEQS = np.load(dname_seq + 'SEQS.npy')
SEQS_TRAINING = np.load(dname_seq + 'SEQS_TRAINING.npy')
sujs = np.load(dname_seq + 'sujs.npy', allow_pickle=True)
n_sujs = len(sujs)
n_dec = SEQS_TRAINING.shape[1]
SEQS.shape

In [ ]:
# Model data
SURPRISE_FITTED = np.load(dname_seq + 'SURPRISE_FITTED.npy')
SURPRISE_FITTED = np.reshape(SURPRISE_FITTED, (n_sujs, -1))

In [ ]:
# EEG files
data_dir = '/media/jacques/DATA1/2019_MusicPred/experimentContinuousMatrix/data/eeg/ERP/'
ch_names = list(np.load(data_dir + 'ch_names.npy'))
ERP = np.load(data_dir + 'aSC_clean.npy')
info = np.load(data_dir + 'info.npy', allow_pickle=True, encoding='latin1').item()
_, n_seqs, n_tones, n_channels, n_times = ERP.shape
ERP.shape

In [ ]:
ERP = ERP.reshape((n_sujs, -1, n_channels, n_times))

In [ ]:
# Model fit
BETAS = np.load('BETAS_FITTED.npy')
R2 = np.load('R2_FITTED.npy')
BETAS.shape

In [ ]:
times = np.linspace(-0.6, 0.55, n_times)

### Function to plot.

In [ ]:
def plotBinsActivity(erp, X, n_bins, ax, title='', var_name='X', color='orangered', tmin=-0.05, tmax=0.4):
    
    '''
    - erp : n_sujs, n_tones, n_times
    - X : n_sujs, n_tones
    '''

    # Compute bins
    frac = 100/n_bins
    percs = np.arange(1, int(100/frac)+1) * frac
    bins = np.percentile(X, percs, axis=1)

    # Bin according to percentiles
    X_bin = np.zeros((n_sujs, n_tones*n_seqs))
    for s, suj in enumerate(range(X.shape[0])):
        X_bin[s] = np.digitize(X[s], bins[:, s])
    
    for j, perc in enumerate(percs):

        # Esthetics
        level = np.linspace(0.05, 1, num=n_bins)[j]
        label = str(perc - frac) + '$ < $' + var_name +'$ \leqslant $ ' + str(perc) + ' %'

        # Get ERp for this surprise bin
        M = np.zeros((n_sujs, n_times))
        for s, suj in enumerate(range(X.shape[0])):
            M[s] = np.nanmean(erp[s, X_bin[s] == j], axis=0)
        m = np.nanmean(M, axis=0)
        se = np.nanstd(M, axis=0) / np.sqrt(X.shape[0])

        # Plot ERP
        ax.fill_between(times, m+se, m-se, alpha=level/(n_bins+1), color=color)
        ax.plot(times, m, alpha=level, color=color, label=label)

    # Esthetics
    ax.set_xlim(tmin, tmax)
    ax.axvline(0, color='k', linestyle='--', alpha=0.6)
    ax.axhline(0, color='k', alpha=0.6)
    ax.set_title(title, fontweight='bold')

### Clustering in time.

In [ ]:
TIMES_SIG = np.zeros((2, n_channels, n_times), dtype=np.bool)
alpha = 0.05
for i, musi in enumerate(np.unique(MUSICIANS)):
    
    # Set threshold
    threshold = stats.distributions.t.ppf(1 - alpha, np.sum(MUSICIANS[:, 0, 0] == musi) - 1)
    
    for j, ch in enumerate(range(n_channels)):
        X = BETAS[MUSICIANS[:, 0, 0] == musi, j]
        t_obs, clusters, pval, H0 = permutation_cluster_1samp_test(
            X, threshold=threshold, n_permutations=2048, tail=0, verbose=False,
        )
        for k, cluster in enumerate(clusters):
            if pval[k] > alpha: continue
            print(pval[k])
            TIMES_SIG[i, j, cluster[0]] = True

TIMES_SIG.shape

In [ ]:
TIMES_DIFF = np.zeros((n_channels, n_times), dtype=np.bool)
alpha = 0.05
threshold = stats.distributions.t.ppf(1 - alpha, np.sum(MUSICIANS[:, 0, 0] == musi) - 1)

for j, ch in enumerate(range(n_channels)):
    X1 = BETAS[MUSICIANS[:, 0, 0] == 1, j]
    X2 = BETAS[MUSICIANS[:, 0, 0] == 0, j]
    t_obs, clusters, pval, H0 = permutation_cluster_test(
        [X1, X2], threshold=threshold, n_permutations=2048, tail=0, verbose=False,
    )
    for k, cluster in enumerate(clusters):
        if pval[k] > alpha: continue
        print(pval[k])
        TIMES_DIFF[j, cluster[0]] = True

### Plot regression on fitted model surprise.

In [ ]:
n_channels = 1

In [ ]:
# Declare figure
fig, AXS = plt.subplots(n_channels, 3, figsize=(21.5, 4*n_channels), facecolor='w')
AXS = np.atleast_2d(AXS)

# Plot betas
for i, axs in enumerate(AXS):
    
    # ERPs
    for j, (musi, color, ax) in enumerate(zip(np.unique(MUSICIANS), colors, axs[:-1])):
        erp = ERP[MUSICIANS[:, 0, 0] == musi][:, :, i].copy()
        x = SURPRISE_FITTED[MUSICIANS[:, 0, 0] == musi]
        title = 'Musicians' if musi == True else 'Non-musicians'
        plotBinsActivity(erp, x, n_bins, ax, color=color, title=title)
        ax.fill_between(times, -10, 10, where=TIMES_SIG[j, i], color='k', alpha=0.05)
        ax.set_ylim(-0.7, 0.7)
        
    # Betas of regression
    ax = axs[-1]
    for j, (musi, color) in enumerate(zip(np.unique(MUSICIANS), colors)):
        m = np.mean(BETAS[MUSICIANS[:, 0, 0] == musi, i], axis=0)
        se = np.std(BETAS[MUSICIANS[:, 0, 0] == musi, i], axis=0)/np.sqrt(np.sum(MUSICIANS[:, 0, 0] == musi))
        label = 'Musicians' if musi == True else 'Non-musicians'
        ax.fill_between(times, m+se, m-se, color=color, alpha=0.1)
        ax.fill_between(times, -10, 10, where=TIMES_DIFF[i], color='k', alpha=0.05)
        ax.plot(times, m, color=color, label=label)
        ax.set_ylim(-1.2, 1.2)
        ax.set_title('Linear regression coefficients SC #' + str(i+1), fontweight='bold')
    ax.legend()
    
    # Esthetics
    for ax in axs:
        ax.set_xlim(tmin, tmax)
        ax.axvline(0, color='k', linestyle='--', alpha=.6)
        ax.axhline(0, color='k', alpha=.6)
plt.tight_layout()
fig

In [ ]:
# Declare figure
fig, AXS = plt.subplots(n_channels, 3, figsize=(21.5, 4*n_channels), facecolor='w')
AXS = np.atleast_2d(AXS)

# Plot betas
for i, axs in enumerate(AXS):
    
    # ERPs
    for j, (musi, color, ax) in enumerate(zip(np.unique(MUSICIANS), colors, axs[:-1])):
        erp = ERP[MUSICIANS[:, 0, 0] == musi][:, :, i].copy()
        x = SURPRISE_FITTED[MUSICIANS[:, 0, 0] == musi]
        title = 'Musicians' if musi == True else 'Non-musicians'
        plotBinsActivity(erp, x, n_bins, ax, color=color, title=title)
        ax.fill_between(times, -10, 10, where=TIMES_SIG[j, i], color='k', alpha=0.05)
        ax.set_ylim(-0.7, 0.7)
        
    # Betas of regression
    ax = axs[-1]
    for j, (musi, color) in enumerate(zip(np.unique(MUSICIANS), colors)):
        m = np.mean(R2[MUSICIANS[:, 0, 0] == musi, i], axis=0)
        se = np.std(R2[MUSICIANS[:, 0, 0] == musi, i], axis=0)/np.sqrt(np.sum(MUSICIANS[:, 0, 0] == musi))
        label = 'Musicians' if musi == True else 'Non-musicians'
        ax.fill_between(times, m+se, m-se, color=color, alpha=0.1)
        ax.fill_between(times, -10, 10, where=TIMES_DIFF[i], color='k', alpha=0.05)
        ax.plot(times, m, color=color, label=label)
        ax.set_ylim(0, 0.015)
        ax.set_title('Linear regression coefficients SC #' + str(i+1), fontweight='bold')
    ax.legend()
    
    # Esthetics
    for ax in axs:
        ax.set_xlim(tmin, tmax)
        ax.axvline(0, color='k', linestyle='--', alpha=.6)
        ax.axhline(0, color='k', alpha=.6)
plt.tight_layout()
fig

### Plot model K0, K1 and K2.

In [ ]:
# Model data
SURPRISE = np.load(dname_seq + 'SURPRISE.npy')
SURPRISE = np.reshape(SURPRISE, (3, n_sujs, -1))
SURPRISE.shape

In [ ]:
# Model fit
BETAS_K = np.load('BETAS_SURPRISE.npy')
R2_K = np.load('R2_SURPRISE.npy')
BETAS_K.shape

In [ ]:
TIMES_SIG_K = np.zeros((3, 2, n_channels, n_times), dtype=np.bool)
alpha = 0.05
for i in range(3):
    for j, musi in enumerate(np.unique(MUSICIANS)):

        # Set threshold
        threshold = stats.distributions.t.ppf(1 - alpha, np.sum(MUSICIANS[:, 0, 0] == musi) - 1)

        for k, ch in enumerate(range(n_channels)):
            X = BETAS_K[i, MUSICIANS[:, 0, 0] == musi, k]
            t_obs, clusters, pval, H0 = permutation_cluster_1samp_test(
                X, threshold=threshold, n_permutations=2048, tail=0, verbose=False,
            )
            for l, cluster in enumerate(clusters):
                if pval[l] > alpha: continue
                print(i, musi)
                TIMES_SIG_K[i, j, k, cluster[0]] = True

TIMES_SIG_K.shape

In [ ]:
TIMES_DIFF_K = np.zeros((3, n_channels, n_times), dtype=np.bool)
alpha = 0.05
threshold = stats.distributions.t.ppf(1 - alpha, n_sujs - 1)

for i in range(3):
    for j, ch in enumerate(range(n_channels)):
        X1 = BETAS_K[i, MUSICIANS[:, 0, 0] == 1, j]
        X2 = BETAS_K[i, MUSICIANS[:, 0, 0] == 0, j]
        
        # Corrected
        t_obs, clusters, pval, H0 = permutation_cluster_test(
            [X1, X2], threshold=threshold, n_permutations=2**11, tail=0, verbose=False,
        )
        for k, cluster in enumerate(clusters):
            if pval[k] > 0.05: continue
            print(pval[k])
            TIMES_DIFF_K[i, j, cluster[0]] = True

In [ ]:
# Declare figure
K = 0
fig, AXS = plt.subplots(n_channels, 3, figsize=(15, 2.5*n_channels), facecolor='w')
AXS = np.atleast_2d(AXS)

# Plot betas
for i, axs in enumerate(AXS):
    
    # ERPs
    for j, (musi, color, ax) in enumerate(zip(np.unique(MUSICIANS), colors, axs[1:])):
        erp = ERP[MUSICIANS[:, 0, 0] == musi][:, :, i].copy()
        x = SURPRISE[K, MUSICIANS[:, 0, 0] == musi]
        title = 'Musicians' if musi == True else 'Non-musicians'
        plotBinsActivity(erp, x, n_bins, ax, color=color, title=title)
        ax.set_ylim(-0.7, 0.7)
        
    # Betas of regression
    ax = axs[0]
    for j, (musi, color) in enumerate(zip(np.unique(MUSICIANS), colors)):
        m = np.mean(BETAS_K[K, MUSICIANS[:, 0, 0] == musi, i], axis=0)
        se = np.std(BETAS_K[K, MUSICIANS[:, 0, 0] == musi, i], axis=0)/np.sqrt(np.sum(MUSICIANS[:, 0, 0] == musi))
        label = 'Musicians' if musi == True else 'Non-musicians'
        ax.fill_between(times, m+se, m-se, color=color, alpha=0.1)
        ax.plot(times, m, color=color, label=label)
        ax.set_ylim(-1.2, 1.2)
        ax.set_title('Linear regression coefficients SC #' + str(i+1), fontweight='bold')
    ax.legend()
    
    # Esthetics
    for ax in axs:
        ax.set_xlim(tmin, tmax)
        ax.axvline(0, color='k', linestyle='--', alpha=.6)
        ax.axhline(0, color='k', alpha=.6)
plt.tight_layout()
fig

In [ ]:
# Declare figure
K = 1
fig, AXS = plt.subplots(n_channels, 3, figsize=(21.5, 4*n_channels), facecolor='w')
AXS = np.atleast_2d(AXS)

# Plot betas
for i, axs in enumerate(AXS):
    
    # ERPs
    for j, (musi, color, ax) in enumerate(zip(np.unique(MUSICIANS), colors, axs[:-1])):
        erp = ERP[MUSICIANS[:, 0, 0] == musi][:, :, i].copy()
        x = SURPRISE[K, MUSICIANS[:, 0, 0] == musi]
        title = 'Musicians' if musi == True else 'Non-musicians'
        plotBinsActivity(erp, x, n_bins, ax, color=color, title=title)
        ax.set_ylim(-0.7, 0.7)
    
    # Betas of regression
    ax = axs[-1]
    for j, (musi, color) in enumerate(zip(np.unique(MUSICIANS), colors)):
        m = np.mean(BETAS_K[K, MUSICIANS[:, 0, 0] == musi, i], axis=0)
        se = np.std(BETAS_K[K, MUSICIANS[:, 0, 0] == musi, i], axis=0)/np.sqrt(np.sum(MUSICIANS[:, 0, 0] == musi))
        label = 'Musicians' if musi == True else 'Non-musicians'
        ax.fill_between(times, m+se, m-se, color=color, alpha=0.1)
        ax.plot(times, m, color=color, label=label)
        ax.set_ylim(-1.2, 1.2)
        ax.set_title('Linear regression coefficients SC #' + str(i+1), fontweight='bold')
    ax.legend()
    
    # Esthetics
    for ax in axs:
        ax.set_xlim(tmin, tmax)
        ax.axvline(0, color='k', linestyle='--', alpha=.6)
        ax.axhline(0, color='k', alpha=.6)
plt.tight_layout()
fig

In [ ]:
# Declare figure
K = 2
fig, AXS = plt.subplots(n_channels, 3, figsize=(15, 2.5*n_channels), facecolor='w')
AXS = np.atleast_2d(AXS)

# Plot betas
for i, axs in enumerate(AXS):
    
    # ERPs
    for j, (musi, color, ax) in enumerate(zip(np.unique(MUSICIANS), colors, axs[1:])):
        erp = ERP[MUSICIANS[:, 0, 0] == musi][:, :, i].copy()
        x = SURPRISE[K, MUSICIANS[:, 0, 0] == musi]
        title = 'Musicians' if musi == True else 'Non-musicians'
        plotBinsActivity(erp, x, n_bins, ax, color=color, title=title)
        ax.set_ylim(-0.7, 0.7)
    
    # Betas of regression
    ax = axs[0]
    for j, (musi, color) in enumerate(zip(np.unique(MUSICIANS), colors)):
        m = np.mean(BETAS_K[K, MUSICIANS[:, 0, 0] == musi, i], axis=0)
        se = np.std(BETAS_K[K, MUSICIANS[:, 0, 0] == musi, i], axis=0)/np.sqrt(np.sum(MUSICIANS[:, 0, 0] == musi))
        label = 'Musicians' if musi == True else 'Non-musicians'
        ax.fill_between(times, m+se, m-se, color=color, alpha=0.1)
        ax.plot(times, m, color=color, label=label)
        ax.set_ylim(-1.2, 1.2)
        ax.set_title('Linear regression coefficients SC #' + str(i+1), fontweight='bold')
    ax.legend()
    
    # Esthetics
    for ax in axs:
        ax.set_xlim(tmin, tmax)
        ax.axvline(0, color='k', linestyle='--', alpha=.6)
        ax.axhline(0, color='k', alpha=.6)
plt.tight_layout()
fig

In [ ]:
# Declare figure
fig, AXS = plt.subplots(3, 3, figsize=(21.5, 12), facecolor='w')
AXS = np.atleast_2d(AXS)

# Plot betas
for K, axs in enumerate(AXS):
    
    # ERPs
    for j, (musi, color, ax) in enumerate(zip(np.unique(MUSICIANS), colors, axs[:-1])):
        erp = ERP[MUSICIANS[:, 0, 0] == musi][:, :, 0].copy()
        x = SURPRISE[K, MUSICIANS[:, 0, 0] == musi]
        title = 'Musicians' if musi == True else 'Non-musicians'
        plotBinsActivity(erp, x, n_bins, ax, color=color, title=title)
        ax.set_ylim(-0.7, 0.7)
        
    # Betas of regression
    ax = axs[-1]
    for j, (musi, color) in enumerate(zip(np.unique(MUSICIANS), colors)):
        m = np.mean(BETAS_K[K, MUSICIANS[:, 0, 0] == musi, 0], axis=0)
        se = np.std(BETAS_K[K, MUSICIANS[:, 0, 0] == musi, 0], axis=0)/np.sqrt(np.sum(MUSICIANS[:, 0, 0] == musi))
        label = 'Musicians' if musi == True else 'Non-musicians'
        ax.fill_between(times, m+se, m-se, color=color, alpha=0.1)
        ax.plot(times, m, color=color, label=label)
        ax.set_ylim(-1.2, 1.2)
        ax.set_title('Linear regression coefficients SC #' + str(0+1), fontweight='bold')
    ax.legend()
    
    # Esthetics
    for ax in axs:
        ax.set_xlim(tmin, tmax)
        ax.axvline(0, color='k', linestyle='--', alpha=.6)
        ax.axhline(0, color='k', alpha=.6)
plt.tight_layout()

In [ ]:
# Declare figure
fig, AXS = plt.subplots(3, 3, figsize=(21.5, 12), facecolor='w')
AXS = np.atleast_2d(AXS)

# Plot betas
for K, axs in enumerate(AXS):
    
    # ERPs
    for j, (musi, color, ax) in enumerate(zip(np.unique(MUSICIANS), colors, axs[:-1])):
        erp = ERP[MUSICIANS[:, 0, 0] == musi][:, :, 0].copy()
        x = SURPRISE[K, MUSICIANS[:, 0, 0] == musi]
        title = 'Musicians' if musi == True else 'Non-musicians'
        plotBinsActivity(erp, x, n_bins, ax, color=color, title=title)
        ax.set_ylim(-0.7, 0.7)
        
    # Betas of regression
    ax = axs[-1]
    for j, (musi, color) in enumerate(zip(np.unique(MUSICIANS), colors)):
        m = np.mean(R2_K[K, MUSICIANS[:, 0, 0] == musi, 0], axis=0)
        se = np.std(R2_K[K, MUSICIANS[:, 0, 0] == musi, 0], axis=0)/np.sqrt(np.sum(MUSICIANS[:, 0, 0] == musi))
        label = 'Musicians' if musi == True else 'Non-musicians'
        ax.fill_between(times, m+se, m-se, color=color, alpha=0.1)
        ax.plot(times, m, color=color, label=label)
        ax.set_ylim(0, 0.015)
        ax.set_title('R2 SC #' + str(0+1), fontweight='bold')
    ax.legend()
    
    # Esthetics
    for ax in axs:
        ax.set_xlim(tmin, tmax)
        ax.axvline(0, color='k', linestyle='--', alpha=.6)
        ax.axhline(0, color='k', alpha=.6)
plt.tight_layout()
fig

### Interaction K0 * K1 and K0 * K2.

In [ ]:
def stat_fun_2x3(A1B1, A2B1, A3B1, A1B2, A2B2, A3B2):
    "A is between subject and B is within"
    
    import pandas as pd
    from pingouin import mixed_anova
    
    # Build a pandas dataframe
    df = pd.DataFrame()
    for (musi, K, data) in  zip([0, 0, 0, 1, 1, 1], 
                                [0, 1, 2, 0, 1, 2], 
                                [A1B1, A2B1, A3B1, A1B2, A2B2, A3B2]):
        _sujs = sujs[MUSICIANS[:, 0, 0] == musi]
        _df = pd.DataFrame(data=data)
        _df['suj'] = _sujs
        _df['K'] = K
        _df['musi'] = musi
        df = pd.concat([df, _df], ignore_index=True)
    
    # Run pingouin ANOVA
    F = np.zeros(A1B1.shape[1])
    for i in range(A1B1.shape[1]):
        aov = mixed_anova(dv=i, between='musi',
                          within='K', subject='suj', data=df)
        F[i] = np.array(aov['F'])[-1]
    
    # Return F-value
    return F

In [ ]:
k = 0
threshold = mne.stats.f_threshold_mway_rm(n_sujs, [2, 3], effects='A:B', pvalue=0.05)
X = [BETAS_K[0, MUSICIANS[:, 0, 0] == 0, k],
     BETAS_K[1, MUSICIANS[:, 0, 0] == 0, k],
     BETAS_K[2, MUSICIANS[:, 0, 0] == 0, k],
     BETAS_K[0, MUSICIANS[:, 0, 0] == 1, k],
     BETAS_K[1, MUSICIANS[:, 0, 0] == 1, k], 
     BETAS_K[2, MUSICIANS[:, 0, 0] == 1, k]
    ]
fig, ax = plt.subplots()
ax.plot(times, stat_fun_2x3(*X))
ax.axhline(threshold, color='k')
ax.set_xlim(-0.05, 0.5)
plt.tight_layout()

### Figure 3.

In [ ]:
W, H = np.array([5, 8])
fig = plt.figure(facecolor='w', figsize=(W, H), dpi=300)
n_col, n_row = (1, 3)
colors = ['#1f77b4', '#ff7f0e']
letter_size = 15

#########################################################
######################### EEG ###########################
#########################################################

# ERPs
y_sig_scale = 0.02
y_sig_offset = 0.1
for i, (musi, color, letter) in enumerate(zip(np.unique(MUSICIANS), colors, ('A', 'B'))):

    # Declare axis
    ax = plt.subplot2grid((n_row, n_col), (i, 0))
    
    # Plot
    erp = - ERP[MUSICIANS[:, 0, 0] == musi][:, :, 0].copy() # Adjust sign
    x = SURPRISE_FITTED[MUSICIANS[:, 0, 0] == musi]
    title = 'Musicians' if musi == True else 'Non-musicians'
    plotBinsActivity(erp, x, n_bins, ax, color=color, title=title, var_name='$-log(P)$')
    ax.fill_between(times, -10, 10, where=TIMES_SIG[i, 0], color=color, alpha=0.05)
    ax.set_ylim(-0.5, 0.7)
    
    # Add significance bars
    y_range = ax.get_ylim()[1] - ax.get_ylim()[0]
    y_sig = ax.get_ylim()[0] + y_range * y_sig_offset
    y_sig_diff = y_range * y_sig_scale
    ax.fill_between(times, y_sig, y_sig + y_sig_diff, where=TIMES_SIG[i, 0], color=color, alpha=1)
    
    # Esthetics
    ax.set_xlim(tmin, tmax)
    ax.axvline(0, color='k', linestyle='--', alpha=.6)
    ax.axhline(0, color='k', alpha=.6)
    ax.set_ylabel('Amplitude (a.u.)')
    ax.legend(fontsize='xx-small', loc=1)
    ax.text(-0.14, 1.05, letter, transform=ax.transAxes, fontsize=letter_size, fontweight='bold')


#########################################################
######################## BETAS ##########################
#########################################################

# Declare axis
ax = plt.subplot2grid((n_row, n_col), (n_row-1, 0)) 

# Plot
for j, (musi, color) in enumerate(zip(np.unique(MUSICIANS), colors)):
    m = - np.mean(BETAS[MUSICIANS[:, 0, 0] == musi, 0], axis=0)
    se = np.std(BETAS[MUSICIANS[:, 0, 0] == musi, 0], axis=0)/np.sqrt(np.sum(MUSICIANS[:, 0, 0] == musi))
    label = 'Musicians' if musi == True else 'Non-musicians'
    ax.fill_between(times, m+se, m-se, color=color, alpha=0.1)
    ax.plot(times, m, color=color, label=label)
    ax.set_ylim(-1.2, 1.2)
    ax.set_title('Linear regression coefficients', fontweight='bold')
    
    # Add significance bars
    y_range = ax.get_ylim()[1] - ax.get_ylim()[0]
    y_sig = ax.get_ylim()[0] + y_range * y_sig_offset
    y_sig_diff = y_range * y_sig_scale
    ax.fill_between(times, y_sig + 2 * j * y_sig_diff, y_sig + y_sig_diff + 2 * j * y_sig_diff, where=TIMES_SIG[j, 0], color=color, alpha=1)    
ax.fill_between(times, -10, 10, where=TIMES_DIFF[0], color='k', alpha=0.05)
ax.legend(fontsize='xx-small', loc=1)

# Esthetics
ax.set_xlim(tmin, tmax)
ax.axvline(0, color='k', linestyle='--', alpha=.6)
ax.axhline(0, color='k', alpha=.6)
ax.set_ylabel('Beta (a.u.)')
ax.text(-0.14, 1.05, 'C', transform=ax.transAxes, fontsize=letter_size, fontweight='bold')

plt.tight_layout(h_pad=3, w_pad=3)

In [ ]:
fname = '/media/jacques/DATA/2019_MusicPred/experimentContinuousMatrix/analysis/Figures/Figure_3.png'
fig.savefig(fname, bbox_inches='tight')

### Fig. Supp. 5

In [ ]:
W, H = np.array([5, 3])
fig = plt.figure(facecolor='w', figsize=(W, H), dpi=300)
n_col, n_row = (1, 1)
colors = ['#1f77b4', '#ff7f0e']
letter_size = 15

#########################################################
########################## R2 ###########################
#########################################################

# Declare axis
ax = plt.subplot2grid((n_row, n_col), (0, 0)) 

# Plot
for j, (musi, color) in enumerate(zip(np.unique(MUSICIANS), colors)):
    m = np.mean(R2[MUSICIANS[:, 0, 0] == musi, 0], axis=0)
    se = np.std(R2[MUSICIANS[:, 0, 0] == musi, 0], axis=0)/np.sqrt(np.sum(MUSICIANS[:, 0, 0] == musi))
    label = 'Musicians' if musi == True else 'Non-musicians'
    ax.fill_between(times, m+se, m-se, color=color, alpha=0.1)
    ax.plot(times, m, color=color, label=label)
    ax.set_ylim(0., 0.015)
    ax.set_title(r'Coefficient of determination $R^2$', fontweight='bold')
    
ax.fill_between(times, -1, 1, where=TIMES_DIFF[0], color='k', alpha=0.05)
ax.legend(fontsize='xx-small', loc=1)

# Esthetics
ax.set_xlim(tmin, tmax)
ax.axvline(0, color='k', linestyle='--', alpha=.6)
ax.axhline(0, color='k', alpha=.6)
ax.set_ylabel(r'$R^2$')

plt.tight_layout(h_pad=3, w_pad=3)
fig

In [ ]:
fname = '/media/jacques/DATA/2019_MusicPred/experimentContinuousMatrix/analysis/Figures/Figure_Supp_5.png'
fig.savefig(fname, bbox_inches='tight')

### Figure 4.

In [ ]:
from bayesianLearner import *
W, H = np.array([15, 8])
fig = plt.figure(facecolor='w', figsize=(W, H), dpi=300)
n_col, n_row = (3, 7)
colors = ['#1f77b4', '#ff7f0e']
letter_size = 15
N_VOC = 3

#########################################################
####################### Matrices ########################
#########################################################

for i, (K, letter) in enumerate(zip(range(3), ('A', 'B', 'C'))): 
    
    # Declare axis
    ax = plt.subplot2grid((n_row, n_col), (0, i), rowspan=1, colspan=1)
    
    # Compute model posterior
    seq = SEQS[0, 0]
    _seq = np.zeros((n_tones, N_VOC))
    _seq[np.arange(n_tones), np.array(seq, dtype=np.int)] = 1
    _, _P, _, _, _ = bayesianHierarchicalInverter(_seq[10:20+10].copy(), K=K)

    # Plot matrix
    ax.imshow(_P[-1].T + np.random.normal(0, 0.05, size=_P[-1].T.shape), 
              vmin=0, vmax=1, cmap='Greys', aspect='equal')
    
    # Esthetics
    ax.set_yticks([])
    ax.set_xticks([])
    ax.set_title('Model K = ' + str(K))
    ax.text(0.03 + i * 0.33, 0.94, letter, transform=fig.transFigure, fontsize=letter_size, fontweight='bold')

#########################################################
######################### ERPs ##########################
#########################################################

# ERPs
y_sig_scale = 0.02
y_sig_offset = 0.1
for i, K in enumerate(range(3)):
    for j, (musi, color, letter) in enumerate(zip(np.unique(MUSICIANS), colors, ('A', 'B'))):

        # Declare axis
        ax = plt.subplot2grid((n_row, n_col), (1+j*2, i), rowspan=2)

        # Plot
        erp = - ERP[MUSICIANS[:, 0, 0] == musi][:, :, 0].copy() # Adjust sign
        x = SURPRISE[K, MUSICIANS[:, 0, 0] == musi]
        title = 'Musicians' if musi == True else 'Non-musicians'
        plotBinsActivity(erp, x, n_bins, ax, color=color, title=title, var_name='$-log(P)$')
        ax.fill_between(times, -10, 10, where=TIMES_SIG_K[i, j, 0], color=color, alpha=0.05)
        ax.set_ylim(-0.5, 0.7)

        # Add significance bars
        y_range = ax.get_ylim()[1] - ax.get_ylim()[0]
        y_sig = ax.get_ylim()[0] + y_range * y_sig_offset
        y_sig_diff = y_range * y_sig_scale
        ax.fill_between(times, y_sig, y_sig + y_sig_diff, where=TIMES_SIG_K[i, j, 0], color=color, alpha=1)

        # Esthetics
        ax.set_xlim(tmin, tmax)
        ax.axvline(0, color='k', linestyle='--', alpha=.6)
        ax.axhline(0, color='k', alpha=.6)
        ax.set_ylabel('Amplitude (a.u.)')
        #ax.legend(fontsize='xx-small', loc=1)

plt.tight_layout()

#########################################################
######################## BETAS ##########################
#########################################################

# Plot betas
for i, (K, letter, ylim) in enumerate(zip(range(3), ('A', 'B', 'C'), [[-3.1, 1.5], [-0.3, 0.65], [-0.3, 0.65]])):
    
    # Declare axis
    ax = plt.subplot2grid((n_row, n_col), (5, i), rowspan=2) 

    # Plot
    for j, (musi, color) in enumerate(zip(np.unique(MUSICIANS), colors)):
        m = - np.mean(BETAS_K[K, MUSICIANS[:, 0, 0] == musi, 0], axis=0)
        se = np.std(BETAS_K[K, MUSICIANS[:, 0, 0] == musi, 0], axis=0)/np.sqrt(np.sum(MUSICIANS[:, 0, 0] == musi))
        label = 'Musicians' if musi == True else 'Non-musicians'
        ax.fill_between(times, m+se, m-se, color=color, alpha=0.1)

        ax.plot(times, m, color=color, label=label)
        ax.set_ylim(ylim)
        ax.set_title('Linear coefficients for model K=' + str(K), fontweight='bold')
        
        # Add significance bars
        y_range = ax.get_ylim()[1] - ax.get_ylim()[0]
        y_sig = ax.get_ylim()[0] + y_range * y_sig_offset
        y_sig_diff = y_range * y_sig_scale
        ax.fill_between(times, y_sig + 2 * j * y_sig_diff, y_sig + y_sig_diff + 2 * j * y_sig_diff, where=TIMES_SIG_K[i, j, 0], color=color, alpha=1)

    # Add interaction significance bars
    ax.fill_between(times, y_sig - 3 * y_sig_diff, y_sig + y_sig_diff - 3 * y_sig_diff, 
                    where=TIMES_INTERACTION, color='k', alpha=1)
    ax.fill_between(times, -10, 10, where=TIMES_DIFF_K[i, 0], color='k', alpha=0.05)
    ax.legend()

    # Esthetics
    ax.set_xlim(tmin, tmax)
    ax.axvline(0, color='k', linestyle='--', alpha=.6)
    ax.axhline(0, color='k', alpha=.6)
    ax.set_ylabel('Beta (a.u.)')

plt.tight_layout(h_pad=1, w_pad=3)

In [ ]:
fname = '/media/jacques/DATA1/2019_MusicPred/experimentContinuousMatrix/analysis/Figures/Figure_4.png'
fig.savefig(fname, bbox_inches='tight')

### Fig. Supp. 6

In [ ]:
W, H = np.array([5, 8])
fig = plt.figure(facecolor='w', figsize=(W, H), dpi=300)
n_col, n_row = (1, 3)
colors = ['#1f77b4', '#ff7f0e']
letter_size = 15

#########################################################
######################## BETAS ##########################
#########################################################

# Plot betas
for i, (K, letter) in enumerate(zip(range(3), ('A', 'B', 'C'))):
    
    # Declare axis
    ax = plt.subplot2grid((n_row, n_col), (i, 0)) 

    # Plot
    for j, (musi, color) in enumerate(zip(np.unique(MUSICIANS), colors)):
        m = np.mean(R2_K[K, MUSICIANS[:, 0, 0] == musi, 0], axis=0)
        se = np.std(R2_K[K, MUSICIANS[:, 0, 0] == musi, 0], axis=0)/np.sqrt(np.sum(MUSICIANS[:, 0, 0] == musi))
        label = 'Musicians' if musi == True else 'Non-musicians'
        ax.fill_between(times, m+se, m-se, color=color, alpha=0.1)

        ax.plot(times, m, color=color, label=label)
        ax.set_ylim(0, 0.015)
        ax.set_title(r'Coeficient of determination $R^2$ for model K=' + str(K), fontweight='bold')

    ax.fill_between(times, -10, 10, where=TIMES_DIFF_K[i, 0], color='k', alpha=0.05)
    ax.legend(fontsize='xx-small', loc=1)

    # Esthetics
    ax.set_xlim(tmin, tmax)
    ax.axvline(0, color='k', linestyle='--', alpha=.6)
    ax.axhline(0, color='k', alpha=.6)
    ax.set_ylabel('$R^2$')
    ax.text(-0.14, 1.05, letter, transform=ax.transAxes, fontsize=letter_size, fontweight='bold')


plt.tight_layout(h_pad=3, w_pad=3)

In [ ]:
fname = '/media/jacques/DATA1/2019_MusicPred/experimentContinuousMatrix/analysis/Figures/Figure_Supp_6.png'
fig.savefig(fname, bbox_inches='tight')